# Demonstration of the funds_py package
Github link: github.com/joaopm33/funds_py <br> 
Author: Joao Penido Monteiro <br> 
Linkedin: linkedin.com/in/joao-penido-monteiro/

## 1. Import the needed libraries

In [91]:
from funds_func import returns, cum_returns, volatility, drawdown, corr_benchmark, beta, alpha, sharpe, sortino, capture_ratio
import pandas as pd
import sqlite3
import seaborn as sns
import plotly.express as px

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## 2. Connect to the database

In [2]:
#connects to the database
con = sqlite3.connect(r"investments_database.db")

## 3. Pull the data from the database to a pandas dataframe
The SQL query bellow can seem a bit complicated at a first glance, but the steps are actually pretty simple: <br>

&ensp;&ensp;1 - Defines a filter that will be used in step 2: we only keep to our dataset funds that had at least 200 clients and 50 million reais in net assets during the month of march 2021. <br>

&ensp;&ensp;2 - Defines our funds table: we select the funds CNPJ (unique identifier), date and daily quota prices. We also join the funds name from the cadastral information table. At this step, a new filter is added: only funds classified in the stocks category are kept. Finally, we keep in our result set only data starting from the year of 2009. <br>

&ensp;&ensp;3 - Defines the selic and ibovespa tables. <br>

&ensp;&ensp;4 - Joins the tables defined in steps 2 and 3, ordering it by date and fund CNPJ.

In [3]:
query = '''
WITH
-- defines the filter table we will use in our query
filter_query AS
(
SELECT DISTINCT CNPJ_FUNDO
FROM daily_quotas
WHERE DT_COMPTC >= '2021-03-01' AND
      NR_COTST >= 200 AND
      VL_PATRIM_LIQ >= 50000000
),

--defines the table with funds daily quotas
funds AS
(SELECT A.*, B.fund_name FROM
(
SELECT DT_COMPTC as date, CNPJ_FUNDO as cnpj_fund, VL_QUOTA as fund_quota
FROM daily_quotas 
WHERE CNPJ_FUNDO IN (select * from filter_query) AND
      DT_COMPTC >= '2009-01-01')A
INNER JOIN 
(
 SELECT CNPJ_FUNDO, DENOM_SOCIAL as fund_name
 FROM info_cadastral_funds
 WHERE julianday('now') - julianday(DT_INI_ATIV) >= 720 AND
 CLASSE = 'Fundo de Ações'
)B
    ON A.cnpj_fund = B.CNPJ_FUNDO
),	

--defines the table with the ibovespa prices
ibov AS
(
SELECT Date, Close AS ibov_price 
FROM ibov_returns
),

-- defines the table with the selic (interest rates) returns
selic AS
(
SELECT date, value as selic_returns
FROM selic_rates
)

SELECT funds.*, ibov.ibov_price, selic.selic_returns from
funds
LEFT JOIN
ibov 
    ON funds.date = ibov.Date
LEFT JOIN
selic
    ON funds.date = selic.date
ORDER BY date, cnpj_fund
'''

In [4]:
funds = pd.read_sql(query, con).drop_duplicates() #makes sure we dont have duplicates

#lets see how our dataset looks like
funds.head(5)

,date,cnpj_fund,fund_quota,fund_name,ibov_price,selic_returns
0,2009-01-02 00:00:00,00.398.561/0001-90,85.693000,PORTO SEGURO AÇÕES FUNDO DE INVESTIMENTO EM CO...,40.32,0.000509
1,2009-01-02 00:00:00,00.575.922/0001-27,750.330490,ALFA AÇÕES PREMIUM - FUNDO DE INVESTIMENTO EM ...,40.32,0.000509
2,2009-01-02 00:00:00,00.601.692/0001-23,1813.325736,FAMA FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS ...,40.32,0.000509
3,2009-01-02 00:00:00,00.822.059/0001-65,5.258319,BB AÇÕES IBOVESPA ATIVO FUNDO DE INVESTIMENTO ...,40.32,0.000509
4,2009-01-02 00:00:00,01.063.897/0001-65,24.736685,ITAÚ SMALL CAP VALUATION FUNDO DE INVESTIMENTO...,40.32,0.000509


## 4. Analyze the data

### returns function
We have prices for each fund and ibovespa, but to actually do any comparation, we need percentual returns. So we'll use the returns function of the funds_py library to calculate them. <br>
Techinical reference: investopedia.com/terms/c/cagr.asp


In [33]:
#Top 5 funds by CAGR
funds_cagr = returns(funds, group = 'cnpj_fund', values = ['fund_quota', 'ibov_price'], rolling = False)
funds_cagr = funds_cagr.merge(funds.groupby('cnpj_fund')[['fund_name']].first(),
                              how = 'left',
                              left_index = True,
                              right_index = True
                             )                     
                         
funds_cagr.sort_values('fund_quota_cagr', ascending = False).head(5)

,fund_quota_cum_return,ibov_price_cum_return,days,fund_quota_cagr,ibov_price_cagr,fund_name
cnpj_fund,,,,,,
24.874.367/0001-00,4.903479,1.005730,1166,0.467758,0.162327,INTER + IBOVESPA ATIVO FUNDO DE INVESTIMENTO E...
29.177.013/0001-12,1.579702,0.358147,738,0.382089,0.110188,TRIGONO FLAGSHIP SMALL CAPS FUNDO DE INVESTIME...
12.332.239/0001-48,3.791291,1.216024,1247,0.372490,0.174452,PILOTIS FUNDO DE INVESTIMENTOS EM AÇÕES
08.968.733/0001-26,1.286935,0.486268,680,0.358736,0.158183,FUNDO DE INVESTIMENTO DE AÇÕES TRIGONO VERBIER
30.129.289/0001-04,1.188318,0.452353,672,0.341348,0.150210,MANAGER ATMOS FUNDO DE INVESTIMENTO EM COTAS D...


### cum_returns function
You can see how the performance of the top 5 funds was build over time.

In [32]:
top5 = (funds_cagr.sort_values('fund_quota_cagr', ascending = False)
         .reset_index()
         .iloc[:5,0] #selects top 5 rows and cnpj column
         .to_list()
        )

top5_returns = cum_returns(funds[funds.cnpj_fund.isin(top5)],
                            group = 'cnpj_fund',
                            values = ['fund_quota', 'ibov_price']
                           )

#plots the top 5 funds performance over time
fig = px.line(top5_returns, 
              x='date', 
              y='fund_quota_cum_return', 
              color = 'fund_name' ,
              title='Top 5 CAGR Funds Performance Over Time',
              labels={"fund_quota_cum_return": "Cumulative return", "fund_name": "Fund name", "date":"date"})


fig.update_layout(legend=dict(
    yanchor="bottom",
    y= -0.7,
    xanchor="left",
    x=0.01
))

fig.show()

### drawdown function
Lets take a look at how underwater (% loss from all time high) each fund in the top 5 was over time. <br>
Techinical reference: investopedia.com/terms/d/drawdown.asp

In [65]:
def drawdown(df: pd.DataFrame, group: str = 'CNPJ_FUNDO', values: list = ['VL_QUOTA'])-> pd.DataFrame:
    """Returns the drawdown (the % the asset is down from its all-time-high) for givens assets.

    Parameters:
    df (pd.DataFrame): Pandas dataframe with the needed data
    group (str): name of the column in the dataframe used to group values. Example: 'stock_ticker' or 'fund_code'
    values (list): names of the columns in the dataframe wich contains the asset and its benchmark prices. Example: ['asset_price', 'index price']. 
   
    Returns:
    pd.DataFrame: The original pandas dataframe with added columns for the all time high and drawdown of the given assets.

   """
    df2 = df.copy(deep = True)
    for value in values:
        col = 'cum_max_'+ value
        df2[col] = df2.groupby([group])[[value]].cummax().to_numpy() 
        df2[('drawdown_'+ value)] = (df2[value]/df2[col])-1
    return df2

In [66]:
top5_drawdown = drawdown(funds[funds.cnpj_fund.isin(top5)],
                            group = 'cnpj_fund',
                            values = ['fund_quota', 'ibov_price']
                           )
#plots the top 5 funds drawdown over time
fig = px.line(top5_drawdown, 
              x='date', 
              y='drawdown_fund_quota', 
              color = 'fund_name' ,
              title='Top 5 CAGR Funds Drawdown Over Time',
              labels={"drawdown_fund_quota": "Drawdown", "fund_name": "Fund name", "date":"date"})


fig.update_layout(legend=dict(
    yanchor="bottom",
    y= -0.7,
    xanchor="left",
    x=0.01
))

fig.show()

### volatility function
Calculates the annualized volatility of each fund <br>
Techinical reference: investopedia.com/terms/v/volatility.asp

In [72]:
##5 most volatile funds

#Calculates daily returns of each fund
funds_returns = returns(funds, group = 'cnpj_fund', values = ['fund_quota', 'ibov_price'], rolling = True)

#calculates the standard deviation of daily returns
funds_vol = volatility(funds_returns, group = 'cnpj_fund', values = ['fund_quota_return_1d', 'ibov_price_return_1d'], rolling = False)

funds_vol = funds_vol.merge(funds.groupby('cnpj_fund')[['fund_name']].first(),
                              how = 'left',
                              left_index = True,
                              right_index = True
                             )                     
                         
funds_vol.sort_values('fund_quota_return_1d_vol', ascending = False).head(5)

,fund_quota_return_1d_vol,ibov_price_return_1d_vol,fund_name
cnpj_fund,,,
31.936.857/0001-41,0.680580,0.330068,TREND IBOVESPA ALAVANCADO FUNDO DE INVESTIMENT...
08.545.330/0001-74,0.591396,0.259150,JOULE VALUE FUNDO DE INVESTIMENTO EM AÇÕES
29.197.177/0001-01,0.546660,0.308811,ITAÚ AÇÕES PETROBRAS I - FUNDO DE INVESTIMENTO
30.518.554/0001-46,0.531615,0.320463,BB AÇÕES PETROBRAS I FUNDO DE INVESTIMENTO
26.648.868/0001-96,0.497781,0.281014,ALASKA BLACK FUNDO DE INVESTIMENTO EM COTAS DE...


### corr_benchmark function
how correlated are our top 5 CAGR funds in comparisson to the ibovespa index? <br>
Techinical reference: investopedia.com/terms/c/correlationcoefficient.asp

In [83]:
#calculates the correlation of daily returns
funds_corr = corr_benchmark(funds_returns,'fund_quota_return_1d', 'ibov_price_return_1d', group = 'cnpj_fund'
                            , rolling = False)

funds_corr = funds_corr.merge(funds.groupby('cnpj_fund')[['fund_name']].first(),
                              how = 'left',
                              left_index = True,
                              right_index = True
                             ) 
funds_corr.loc[top5, :].sort_values('correlation_benchmark', ascending = False).head(5)

,correlation_benchmark,fund_name
cnpj_fund,,
30.129.289/0001-04,0.911709,MANAGER ATMOS FUNDO DE INVESTIMENTO EM COTAS D...
29.177.013/0001-12,0.774769,TRIGONO FLAGSHIP SMALL CAPS FUNDO DE INVESTIME...
24.874.367/0001-00,0.768505,INTER + IBOVESPA ATIVO FUNDO DE INVESTIMENTO E...
08.968.733/0001-26,0.693251,FUNDO DE INVESTIMENTO DE AÇÕES TRIGONO VERBIER
12.332.239/0001-48,0.675082,PILOTIS FUNDO DE INVESTIMENTOS EM AÇÕES


### beta function
Having both the volatilities of the funds and their correlations with the ibovespa index, we can easily calculate their betas (measure of the volatility of an asset compared to the market) <br>
Techinical reference: investopedia.com/terms/b/beta.asp

In [98]:
#joins correlation and volatilities dataframes
funds_beta = funds_vol.drop(columns = 'fund_name').join(funds_corr)

#calculates betas
funds_beta = beta(funds_beta, 
                  asset_vol = 'fund_quota_return_1d_vol' , 
                  bench_vol = 'ibov_price_return_1d_vol', 
                  correlation = 'correlation_benchmark')

funds_beta.loc[top5, :].sort_values('beta', ascending = False).head(5)

,fund_quota_return_1d_vol,ibov_price_return_1d_vol,correlation_benchmark,fund_name,beta
cnpj_fund,,,,,
30.129.289/0001-04,0.316348,0.318228,0.911709,MANAGER ATMOS FUNDO DE INVESTIMENTO EM COTAS D...,0.906324
24.874.367/0001-00,0.312238,0.277790,0.768505,INTER + IBOVESPA ATIVO FUNDO DE INVESTIMENTO E...,0.863807
29.177.013/0001-12,0.260065,0.311140,0.774769,TRIGONO FLAGSHIP SMALL CAPS FUNDO DE INVESTIME...,0.647588
08.968.733/0001-26,0.233343,0.316847,0.693251,FUNDO DE INVESTIMENTO DE AÇÕES TRIGONO VERBIER,0.510547
12.332.239/0001-48,0.145635,0.274702,0.675082,PILOTIS FUNDO DE INVESTIMENTOS EM AÇÕES,0.357898


### alpha function
Having the funds, ibovespa and selic returns as well as the funds beta we can calculate how much excess of return each fund manager added when compared to the market. Wich funds beat the market?  <br>
Techinical reference: investopedia.com/terms/a/alpha.asp

In [33]:
#joins correlation and volatilities dataframes
funds_alpha = funds_beta.drop(columns = 'fund_name').join(funds_corr)

#calculates betas
funds_beta = beta(funds_beta, 
                  asset_vol = 'fund_quota_return_1d_vol' , 
                  bench_vol = 'ibov_price_return_1d_vol', 
                  correlation = 'correlation_benchmark')

funds_beta.loc[top5, :].sort_values('beta', ascending = False).head(5)

array(['17.759.778/0001-81', '21.917.184/0001-29', '29.574.952/0001-09'],
      dtype=object)

In [13]:
volatility(fundos,values =['VL_QUOTA_return_1d', 'ibov_return_1d'], rolling = 'False', window_size = 252,
          returns_frequency = 1)

Exception: Wrong Parameter: rolling can only be True or False.